In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('./yonhapall.pkl')

In [3]:
len(df)

971806

In [10]:
df[df.article_len<50].sample(20, random_state=2)

,article,date,section,title,article_len
494201,대경대학교 전경 대경대학교 캠퍼스 전경.,2016-05-17 15:37,사회일반,"대경대, 영남권 '교육 기부 거점대학'에 선정",22
76013,아파트 견본주택 21일 부산시 연제구 한 내부가 사람들로 북적...,2015-06-26 10:32,소비자,부동산 활기에 부산 가계대출 빠르게 증가,45
571102,엔에스브이는 이사회 결의에 따라 진채현 변호사가 대표이사로 선임...,2016-02-11 15:01,기업일반,"엔에스브이, 진채현 대표이사 선임",47
173432,◇ 과장급 전보 ▲ 소재부품정책과장 최규종 ▲ 조선해양...,2015-05-11 10:07,경제일반,<인사> 산업통상자원부,45
672527,"김기춘 조윤선 (CG) 특검 ""김기춘, 조윤선 모두 피의자 신분...",2017-01-16 14:54,검찰,"특검 ""김기춘, 조윤선 모두 피의자 신분 소환""(속보)",47
863002,"(단위:억원,만주) ※ 오후 3시 30분 장마감 기준 ...",2017-04-04 15:58,증권시장,[표] 거래소 외국인 순매수도 상위종목(4일),32
392545,(단위:억원) ※ 오후 3시 30분 장마감 기준 ...,2016-11-15 15:43,증권시장,<표> 투자자별 매매동향(15일),30
532643,화재진압훈련을 하고 있는 신임 소방공무원,2016-04-25 12:02,지방행정,"""소방관 경력채용 합격자 오류, 엑셀 작업 실수 탓""",22
458796,※ 자료 : 각 기관,2016-12-07 12:00,경제일반,<표> 주요기관 2017년 한국 경제성장률 전망치,11
248672,▲ 사장 겸 편집국장 최회봉,2015-03-24 14:00,언론·미디어일반,<인사> 아시아타임즈,15


In [13]:
df = df[df.article_len>50]
df=df.reset_index(drop=True)

## 섹션 분리

In [14]:
df['sec']=df['section'].apply(lambda x: x.replace('일반', ''))

In [15]:
counts=df.groupby('sec').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec
지방행정        114720
사회          102979
국제           86699
정치           81340
경제           80778
야구           23369
기업           19085
산업           17559
문화           15430
중앙행정         14714
축구           14157
스포츠          13850
연예           13508
외교           12804
재테크          11668
국방           11254
대기업          10579
북한            9350
경찰            9201
교육            8966
IT            8959
골프            7842
국회            7448
소비자           6837
편집            5540
농구            5487
금융·증권         5393
대학교육          5055
사건            4729
의료            4489
             ...  
임대업              4
신용카드사            3
장애인올림픽           3
수상경기             3
익스트림게임           3
금시세              2
기업분석             2
우슈               2
교포               2
무속신앙             2
국내대회             2
헤드라인             1
개발원조             1
OANA             1
아동청소년복지          1
유럽연합             1
비치발리볼            1
세팍타크로            1
장애아시안게임          1
기타국내종합경기         1
전국소년체전           1
북한사회    

In [16]:
sorted_counts.to_csv('./yonhapsec.csv')

In [17]:
len(df)

955325

In [18]:
def high_sec(section):
    if section in ('사회', '사건', '사고', '검찰', '법원', '해상사고', '육상사고', '사기', '폭행', '자살') :
        return '사회'
    elif section in ('정치', '북한', '남북교류', '남북회담','청와대') :
        return '정치'
    elif section in ('경제', '증권시장', '금융·증권', '부동산', '해외증시', '자본시장', '경제정책', '금융시장', '외환시장', '수출', 'CEO', '외국계기업', '인수합병', '물가', '유가', '기업실적발표', '부도파산', '기업공시', '채권', '펀드', '금리') :
        return '경제'
    elif section in ('문화', '영화', '전시공연', '문화시설', '축제', '문화정책', '연극', '애니매이션', '무형문화재', '클래식', '무용') :
        return '문화'
    elif section in ('기상', '의학', '날씨', '생활정보', '보건', '생활건강', '식물', '요리·맛집', '건강·웰빙') :
        return '생활'
    elif section in ('과학기술', '항공우주산업', '첨단범죄', '자연과학', '생명공학') :
        return '과학'
    elif section in ('국제', '국제기구') :
        return '국제'
    else :
        return ''
    
        

In [19]:
df['sec1']=df['sec'].apply(high_sec)



In [20]:
sec1_counts=df.groupby('sec1').size()
sec1_sorted_counts=sec1_counts.sort_values(ascending=False)
sec1_sorted_counts

sec1
      512013
사회    116977
경제    108239
정치     96050
국제     86750
문화     24176
생활      8565
과학      2555
dtype: int64

In [21]:
df_cate = df[df['sec1']!='']

In [22]:
df_cate=df_cate.reset_index(drop=True)
len(df_cate)

443312

In [23]:
counts=df_cate.groupby('sec1').size()
section_idx=enumerate(counts.keys())
section_to_int = {word: ii for ii, word in section_idx}
section_to_int

{'경제': 0, '과학': 1, '국제': 2, '문화': 3, '사회': 4, '생활': 5, '정치': 6}

In [24]:
def set_section_idx(section):
    return section_to_int[section]

In [25]:
df_cate['sec_idx']=df_cate['sec1'].apply(set_section_idx)

In [26]:
df_cate.drop(['date', 'section','sec'], axis = 1, inplace = True)

In [27]:
df_cate.to_pickle('yonhap_ready.pkl')

In [211]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]


CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.42 ms


In [212]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [213]:
%%time
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=2, max_df=.25)
vect_weights = vect.fit_transform(docs)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

CPU times: user 26min 27s, sys: 25.8 s, total: 26min 52s
Wall time: 26min 53s


In [214]:
weights_df.sort_values(by='weight', ascending=False).head(50)

,term,weight
2722259,대통령,0.008269
9440335,중국,0.008066
3664119,미국,0.006787
7674869,의원,0.005966
4573158,북한,0.005923
2764477,대표,0.005839
8887803,정부,0.005522
782675,경찰,0.005497
6120161,시장,0.005140
11470229,한국,0.004989
